<a href="https://colab.research.google.com/github/deepakra0/tf/blob/Preprocessing/tf_BC_kp_binclass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import pandas as pd
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer/breast-cancer.data'
df = pd.read_csv(url
                 ,na_values='?'
                 ,names = ['recurrence', 'age', 'menopause', 'tumorsize', 'invnodes', 'nodecaps', 'degmalig', 'breast', 'breastquad' , 'irradiat']
)
df.dropna(inplace=True)

In [2]:
ds = tf.data.Dataset.from_tensor_slices((dict(df.iloc[:,1:]), df.iloc[:,0].replace({'no-recurrence-events': 0, 'recurrence-events': 1})))
ds = ds.shuffle(buffer_size=len(df))
ds = ds.batch(10)

In [20]:
ds_feature,ds_target = next(iter(ds))

In [93]:
n1 = tf.keras.layers.experimental.preprocessing.StringLookup()

In [94]:
pf1 = ds.map(lambda x,y:x['age'])

In [95]:
pf1 = pf1.map(lambda x: tf.expand_dims(x, -1))

In [96]:
n1.adapt(pf1)

In [104]:
n1(tf.keras.Input(shape=(1,),name='age', dtype=tf.string))

<KerasTensor: shape=(None, 1) dtype=int64 (created by layer 'string_lookup_3')>

In [23]:
l1 = tf.keras.layers.Input(shape=(1,),name='degamlig', dtype=tf.int32)

In [25]:
n1(l1)(ds_feature['degmalig'])

TypeError: ignored

In [26]:
def encode_numerical_feature(layer, name, dataset):
    # Prepare a Dataset that only yields our feature
    feature = dataset.map(lambda x, y: x[name])
    feature = feature_ds.map(lambda x: tf.expand_dims(x, -1))

    # Learn the statistics of the data
    normalizer = tf.keras.layers.experimental.preprocessing.Normalization()
    normalizer.adapt(feature)

    # Normalize the input feature
    return normalizer(layer)

In [48]:
for spec in ds.element_spec:
  print(spec.keys())

dict_keys(['age', 'menopause', 'tumorsize', 'invnodes', 'nodecaps', 'degmalig', 'breast', 'breastquad', 'irradiat'])


AttributeError: ignored

In [108]:
input_layers = []
encoded_features = []
for key in ds.element_spec[0].keys():
  feature = ds.map(lambda x, y: x[key])
  feature = feature.map(lambda x: tf.expand_dims(x, -1))
  if ds.element_spec[0][key].dtype == 'string':
    categorize = tf.keras.layers.experimental.preprocessing.StringLookup(output_mode='binary')
    categorize.adapt(feature)
    tmp_layer = tf.keras.Input(shape=(1,),name=key,dtype=tf.string)
    encoded_features.append(categorize(tmp_layer))
    input_layers.append(tmp_layer)
  elif ds.element_spec[0][key].dtype == 'int64':
    normalizer = tf.keras.layers.experimental.preprocessing.Normalization()
    normalizer.adapt(feature)
    tmp_layer = tf.keras.Input(shape=(1,),name=key)
    encoded_features.append(normalizer(tmp_layer))
    input_layers.append(tmp_layer)


In [117]:
concat_feature = tf.keras.layers.concatenate(encoded_features)

In [118]:
x = tf.keras.layers.Dense(32,activation='relu')(concat_feature)

In [123]:
output = tf.keras.layers.Dense(1,activation='sigmoid')(x)

In [124]:
model = tf.keras.Model(input_layers,output)

In [125]:
model.compile("adam", "binary_crossentropy", metrics=["accuracy"])

In [130]:
history = model.fit(ds,epochs=50,validation_data=ds)

Epoch 1/50
28/28 [==============================] - 0s 15ms/step - loss: 0.2626 - accuracy: 0.9206 - val_loss: 0.2543 - val_accuracy: 0.9242
Epoch 2/50
28/28 [==============================] - 0s 4ms/step - loss: 0.2576 - accuracy: 0.9242 - val_loss: 0.2524 - val_accuracy: 0.9206
Epoch 3/50
28/28 [==============================] - 0s 4ms/step - loss: 0.2588 - accuracy: 0.9170 - val_loss: 0.2488 - val_accuracy: 0.9242
Epoch 4/50
28/28 [==============================] - 0s 4ms/step - loss: 0.2523 - accuracy: 0.9242 - val_loss: 0.2465 - val_accuracy: 0.9278
Epoch 5/50
28/28 [==============================] - 0s 5ms/step - loss: 0.2497 - accuracy: 0.9278 - val_loss: 0.2442 - val_accuracy: 0.9278
Epoch 6/50
28/28 [==============================] - 0s 5ms/step - loss: 0.2480 - accuracy: 0.9278 - val_loss: 0.2415 - val_accuracy: 0.9242
Epoch 7/50
28/28 [==============================] - 0s 5ms/step - loss: 0.2444 - accuracy: 0.9242 - val_loss: 0.2392 - val_accuracy: 0.9314
Epoch 8/50
28/28 [=